In [1]:
import pandas as pd

In [2]:
df = pd.read_csv(r"cancer_prediction_data (2).csv")

In [3]:
df.head()

,Age,Gender,Tumor_Size,Tumor_Grade,Symptoms_Severity,Family_History,Smoking_History,Alcohol_Consumption,Exercise_Frequency,Cancer_Present
0,62.0,Male,9.763242,High,Mild,Yes,Former Smoker,Moderate,NaN,0
1,33.0,Male,7.946532,NaN,Moderate,Yes,Current Smoker,NaN,Regularly,1
2,33.0,Female,NaN,High,Moderate,No,Current Smoker,High,Rarely,1
3,58.0,Female,NaN,Low,Mild,NaN,Current Smoker,NaN,Rarely,1
4,NaN,Female,1.384231,Medium,Mild,No,Former Smoker,Moderate,Occasionally,1


###  AUTO EDA

In [60]:
!pip install sweetvz

ERROR: Could not find a version that satisfies the requirement sweetvz (from versions: none)
ERROR: No matching distribution found for sweetvz


In [4]:
import sweetviz

In [5]:
report = sweetviz.analyze(df)
report.show_html('eda.html')

                                             |                                             | [  0%]   00:00 ->…

Report eda.html was generated! NOTEBOOK/COLAB USERS: the web browser MAY not pop up, regardless, the report IS saved in your notebook/colab files.


In [4]:
cont = ['Age','Tumor_Size']
ordi = ['Tumor_Grade','Symptoms_Severity','Alcohol_Consumption']
nomi = ['Gender','Family_History','Smoking_History','Exercise_Frequency']

Cont -mean,

ordin-Mode,Ordinal Encoding

nomi -Mode,One Hot Encoding

In [5]:
from sklearn.impute import SimpleImputer

In [6]:
cont_impute = SimpleImputer(strategy='mean')
nomi_impute = SimpleImputer(strategy='most_frequent')
ordi_impute = SimpleImputer(strategy='most_frequent')

In [7]:
df[cont] = cont_impute.fit_transform(df[cont])

In [8]:
df[ordi] = ordi_impute.fit_transform(df[ordi])
df[nomi] = nomi_impute.fit_transform(df[nomi])

In [9]:
df.isna().sum()

Age                    0
Gender                 0
Tumor_Size             0
Tumor_Grade            0
Symptoms_Severity      0
Family_History         0
Smoking_History        0
Alcohol_Consumption    0
Exercise_Frequency     0
Cancer_Present         0
dtype: int64

### Encoding

In [10]:
from sklearn.preprocessing import OrdinalEncoder,OneHotEncoder

In [11]:
df[ordi]

,Tumor_Grade,Symptoms_Severity,Alcohol_Consumption
0,High,Mild,Moderate
1,Low,Moderate,High
2,High,Moderate,High
3,Low,Mild,High
4,Medium,Mild,Moderate
...,...,...,...
1995,High,Mild,Low
1996,High,Moderate,High
1997,Medium,Mild,High
1998,Low,Severe,High


In [12]:
ordinal  = {
    "Tumor_Grade":['Low','Medium','High'],
    'Symptoms_Severity':['Mild', 'Moderate', 'Severe'],
    'Alcohol_Consumption':['None','Low','Moderate','High']
    
}

In [13]:
oe = OrdinalEncoder(categories=[ordinal["Tumor_Grade"],
                                ordinal['Symptoms_Severity'],
                                ordinal['Alcohol_Consumption']],
                   dtype='int')

In [14]:
df[ordi] = oe.fit_transform(df[ordi])

In [15]:
ohe = OneHotEncoder(drop = 'first',sparse_output=True)

In [16]:
df1 = pd.DataFrame(ohe.fit_transform(df[nomi]).toarray(),columns=ohe.get_feature_names_out())

In [17]:
df1

,Gender_Male,Family_History_Yes,Smoking_History_Former Smoker,Smoking_History_Non-Smoker,Exercise_Frequency_Occasionally,Exercise_Frequency_Rarely,Exercise_Frequency_Regularly
0,1.0,1.0,1.0,0.0,0.0,0.0,0.0
1,1.0,1.0,0.0,0.0,0.0,0.0,1.0
2,0.0,0.0,0.0,0.0,0.0,1.0,0.0
3,0.0,1.0,0.0,0.0,0.0,1.0,0.0
4,0.0,0.0,1.0,0.0,1.0,0.0,0.0
...,...,...,...,...,...,...,...
1995,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1996,0.0,1.0,1.0,0.0,1.0,0.0,0.0
1997,0.0,1.0,0.0,1.0,0.0,0.0,0.0
1998,0.0,1.0,1.0,0.0,0.0,1.0,0.0


In [18]:
df.drop(nomi,axis =1 ,inplace = True)

In [19]:
df = pd.concat([df,df1],axis = 1)

In [20]:
df

,Age,Tumor_Size,Tumor_Grade,Symptoms_Severity,Alcohol_Consumption,Cancer_Present,Gender_Male,Family_History_Yes,Smoking_History_Former Smoker,Smoking_History_Non-Smoker,Exercise_Frequency_Occasionally,Exercise_Frequency_Rarely,Exercise_Frequency_Regularly
0,62.000000,9.763242,2,0,2,0,1.0,1.0,1.0,0.0,0.0,0.0,0.0
1,33.000000,7.946532,0,1,3,1,1.0,1.0,0.0,0.0,0.0,0.0,1.0
2,33.000000,5.133781,2,1,3,1,0.0,0.0,0.0,0.0,0.0,1.0,0.0
3,58.000000,5.133781,0,0,3,1,0.0,1.0,0.0,0.0,0.0,1.0,0.0
4,49.267808,1.384231,1,0,2,1,0.0,0.0,1.0,0.0,1.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1995,53.000000,4.058134,2,0,1,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1996,57.000000,5.542532,2,1,3,1,0.0,1.0,1.0,0.0,1.0,0.0,0.0
1997,49.267808,0.866000,1,0,3,0,0.0,1.0,0.0,1.0,0.0,0.0,0.0
1998,47.000000,7.859645,0,2,3,1,0.0,1.0,1.0,0.0,0.0,1.0,0.0


In [21]:
X = df.drop('Cancer_Present',axis =1)
y = df['Cancer_Present']

In [22]:
from sklearn.model_selection import train_test_split


In [23]:
X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.2,random_state=23)

### KNN

In [24]:
from sklearn.metrics import accuracy_score

In [25]:
from sklearn.neighbors import KNeighborsClassifier
knn = KNeighborsClassifier()
knn.fit(X_train,y_train)
y_pred = knn.predict(X_test)
accuracy_score(y_test,y_pred)

0.5225

## BUilding ML MOdel using Pipeline and Column Tranformer

In [26]:
df = pd.read_csv(r"cancer_prediction_data (2).csv")

In [27]:
df['Age'].fillna(df['Age'].mean(),inplace=True)

C:\Users\pc world\AppData\Local\Temp\ipykernel_15816\694922604.py:1: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df['Age'].fillna(df['Age'].mean(),inplace=True)


In [28]:
df.head()

,Age,Gender,Tumor_Size,Tumor_Grade,Symptoms_Severity,Family_History,Smoking_History,Alcohol_Consumption,Exercise_Frequency,Cancer_Present
0,62.000000,Male,9.763242,High,Mild,Yes,Former Smoker,Moderate,NaN,0
1,33.000000,Male,7.946532,NaN,Moderate,Yes,Current Smoker,NaN,Regularly,1
2,33.000000,Female,NaN,High,Moderate,No,Current Smoker,High,Rarely,1
3,58.000000,Female,NaN,Low,Mild,NaN,Current Smoker,NaN,Rarely,1
4,49.267808,Female,1.384231,Medium,Mild,No,Former Smoker,Moderate,Occasionally,1


In [29]:
cont = [2]
ordi = [3,4,7]
nomi = [1,5,6,8]

In [30]:
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler,OneHotEncoder,OrdinalEncoder
from sklearn.impute import SimpleImputer
from sklearn.compose import ColumnTransformer
from sklearn.neighbors import KNeighborsClassifier

In [31]:
num_pipe = Pipeline([("Mean Imputation",SimpleImputer(strategy='mean')),
                     ('Scaling',StandardScaler())
                    ])

nom_pipe = Pipeline([("Mode Imputation",SimpleImputer(strategy='most_frequent')),
         ("Encoder",OneHotEncoder(drop='first'))])

ord_pipe = Pipeline([("Mode Imputation",SimpleImputer(strategy='most_frequent')),
         ("Encoder",OrdinalEncoder())])

In [32]:
preprocess = ColumnTransformer([("Continuous",num_pipe,cont),
                  ('Nominal',nom_pipe,nomi),
                  ('Ordinal',ord_pipe,ordi)],
                              remainder='passthrough')
preprocess

ColumnTransformer(remainder='passthrough',
                  transformers=[('Continuous',
                                 Pipeline(steps=[('Mean Imputation',
                                                  SimpleImputer()),
                                                 ('Scaling',
                                                  StandardScaler())]),
                                 [2]),
                                ('Nominal',
                                 Pipeline(steps=[('Mode Imputation',
                                                  SimpleImputer(strategy='most_frequent')),
                                                 ('Encoder',
                                                  OneHotEncoder(drop='first'))]),
                                 [1, 5, 6, 8]),
                                ('Ordinal',
                                 Pipeline(steps=[('Mode Imputation',
                                                  SimpleImputer(strategy='most_frequent')),
                                                 ('Encoder',
                                                  OrdinalEncoder())]),
                                 [3, 4, 7])])

In [33]:
model = Pipeline([("Data Preproessing",preprocess),
         ("Algorithm ",KNeighborsClassifier())])

In [34]:
model

Pipeline(steps=[('Data Preproessing',
                 ColumnTransformer(remainder='passthrough',
                                   transformers=[('Continuous',
                                                  Pipeline(steps=[('Mean '
                                                                   'Imputation',
                                                                   SimpleImputer()),
                                                                  ('Scaling',
                                                                   StandardScaler())]),
                                                  [2]),
                                                 ('Nominal',
                                                  Pipeline(steps=[('Mode '
                                                                   'Imputation',
                                                                   SimpleImputer(strategy='most_frequent')),
                                                                  ('Encoder',
                                                                   OneHotEncoder(drop='first'))]),
                                                  [1, 5, 6, 8]),
                                                 ('Ordinal',
                                                  Pipeline(steps=[('Mode '
                                                                   'Imputation',
                                                                   SimpleImputer(strategy='most_frequent')),
                                                                  ('Encoder',
                                                                   OrdinalEncoder())]),
                                                  [3, 4, 7])])),
                ('Algorithm ', KNeighborsClassifier())])

In [35]:
X = df.drop('Cancer_Present',axis = 1)
y = df['Cancer_Present']

In [36]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

In [37]:
X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.2,random_state=23)

In [38]:
model.fit(X_train,y_train)
y_pred = model.predict(X_test)
accuracy_score(y_test,y_pred)


0.5525

In [39]:
model.feature_names_in_

array(['Age', 'Gender', 'Tumor_Size', 'Tumor_Grade', 'Symptoms_Severity',
       'Family_History', 'Smoking_History', 'Alcohol_Consumption',
       'Exercise_Frequency'], dtype=object)

In [42]:
!pip install pickle

ERROR: Could not find a version that satisfies the requirement pickle (from versions: none)
ERROR: No matching distribution found for pickle


In [40]:
import pickle

In [41]:
pickle.dump(model,open("estimator_pl.pkl","wb"))

In [42]:
model_pickled = pickle.load(open("estimator_pl.pkl","rb"))